In [69]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

# Start projektu - model sieci neuronowej zdefiniowany obiektowo

budowa modelu opartego na klasie Pythona z użyciem **funkcji specjalnych**: <span style="color:red; font-size:16pt;
font-family:Impact;">init, call i repr</span>

In [70]:
mnist = tf.keras.datasets.mnist

(x_train,y_train),(x_test,y_test) = mnist.load_data()
x_train,x_test = x_train/255.0, x_test/255.0
x_train = x_train[...,tf.newaxis].astype("float32")
x_test = x_test[...,tf.newaxis].astype("float32")

In [71]:
#przygotwanie zbiorów danych do przetwarzania wsadowego dla modelu
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train,y_train)
).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32)

In [72]:
x_train.shape

(60000, 28, 28, 1)

In [73]:
x_test.shape

(10000, 28, 28, 1)

# Budowa modelu w oparciu o interfejs: API do tworzenia podklas modelu KERAS

In [74]:
import matplotlib.pyplot as plt

def relu(x):
    return np.maximum(0, x)

x = np.linspace(-10, 10, 1000)
y = relu(x)

fig,ax = plt.subplots(figsize=[10, 5])

ax.plot(x, y)
ax.set_title('ReLU Function')
ax.set_xlabel('x')
ax.set_ylabel('ReLU(x)')
ax.grid(True)
fig.savefig('relu.png')
plt.show()

<Figure size 720x360 with 1 Axes>

In [75]:
class ModelNeuralNet(Model):
    # def __new__(cls, *args, **kwargs):
    #     return object.__new__(ModelNeuralNet)
    def __init__(self):
        super(ModelNeuralNet,self).__init__()
        self.conv1 = Conv2D(32,3,activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128,activation='relu')
        self.d2 = Dense(10)

    def call(self,x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)
    
    def __repr__(self):
        return  f"klasa {self.__class__.__name__} -> wynik call: połaczone warstwy modelu"

In [76]:
model = ModelNeuralNet()
print(model)

klasa ModelNeuralNet -> wynik call: połaczone warstwy modelu


In [77]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [78]:
#metryki do pomiary straty i dokładności modelu
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [79]:
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [80]:
#użycie tf.GradientTape do trenowania modelu

@tf.function
def train_step(images,labels):
    with tf.GradientTape() as tape:
        predictions = model(images,training=True)
        loss = loss_object(labels,predictions)
    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels,predictions)

In [81]:
@tf.function
def test_step(images,labels):
    predictions = model(images,training=False)
    t_loss = loss_object(labels,predictions)

    test_loss(t_loss)
    test_accuracy(labels,predictions)


# Trenowanie modelu opisanego obiektowo z użyciem funkcji @tf.function

In [82]:
EPOCHS = 5
for epoch in range(EPOCHS):
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_state()

    for images, labels in train_ds:
        train_step(images,labels)

    for test_images, test_labels in test_ds:
        test_step(test_images,test_labels)

    print(
        f'epoch {epoch + 1},'
        f'Loss: {train_loss.result()},'
        f'Accuracy: {train_accuracy.result()*100},'
        f'Test Loss: {test_loss.result()},'
        f'Test Accuracy: {test_accuracy.result()*100}'
    )

epoch 1,Loss: 0.13445931673049927,Accuracy: 95.89666748046875,Test Loss: 0.06309288740158081,Test Accuracy: 97.93000030517578
epoch 2,Loss: 0.04094651713967323,Accuracy: 98.68000030517578,Test Loss: 0.05404546856880188,Test Accuracy: 98.23999786376953
epoch 3,Loss: 0.021835045889019966,Accuracy: 99.28666687011719,Test Loss: 0.05173453316092491,Test Accuracy: 98.3699951171875
epoch 4,Loss: 0.013543202541768551,Accuracy: 99.52832794189453,Test Loss: 0.05220885947346687,Test Accuracy: 98.45999908447266
epoch 5,Loss: 0.008073427714407444,Accuracy: 99.75333404541016,Test Loss: 0.06927244365215302,Test Accuracy: 98.29000091552734
